In [1]:
#Dependencies 
import pandas as pd

In [2]:
#create variable for path of file we will open
filepath = 'data/purchase_data.csv'

In [3]:
df = pd.read_csv(filepath) #create df from csv file
df.count()
df.nunique()
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [4]:
df.nunique()

Purchase ID    780
SN             576
Age             39
Gender           3
Item ID        183
Item Name      179
Price          145
dtype: int64

#  Player Count

Display the total number of players



In [5]:
num_players = df['SN'].nunique() #count the number of unique players in the SN column 
num_players

576

 # Purchasing Analysis (Total)
 
Run basic calculations to obtain number of unique items, average price, etc.


Create a summary data frame to hold the results


Optional: give the displayed data cleaner formatting


Display the summary data frame

In [6]:
num_items = df['Item ID'].nunique() #find number of unique items by item ID
num_items

183

In [7]:
total_revenue = df['Price'].sum() #calculate total revenue
total_revenue

2379.77

In [8]:
num_purchase = df['Purchase ID'].count() #calculate number of purchases
num_purchase

780

In [9]:
df_price = df.drop_duplicates(subset='Item ID')
avg_price = df_price.mean()['Price']
avg_price = round(avg_price, 2)
avg_price

3.04

In [10]:
stats_dictionary = {'Number of Unique Items' : num_items, 'Average Price' : avg_price, 'Number of Purchases' :
                   num_purchase, 'Total Revenue': total_revenue}
pd.DataFrame(stats_dictionary, index=[0])


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,3.04,780,2379.77
